In [1]:
import numpy as np
import torch
from scipy.optimize import least_squares, fsolve

In [2]:
def compute_lambda_sequence(d, phi1, beta1, truncation_size):
    # Инициализация списков для хранения delta и lambda
    delta = [1.0]  # delta_{d,0} = 1
    lambdas = []
    
    # Вычисление lambda_1
    lambda1 = phi1 - beta1 + d
    lambdas.append(lambda1)
    
    # Вычисление delta_{d,1} (если требуется)
    if truncation_size >= 2:
        delta1 = (1 - d) / 1  # delta_{d,1} = (1 - d)/1
        delta.append(delta1)
    
    # Вычисление lambda_2
    if truncation_size >= 2:
        lambda2 = (d - beta1) * (beta1 - phi1) + d * (1 - d) / 2
        lambdas.append(lambda2)
    
    # Вычисление delta_{d,k} и lambda_k для k >= 3
    for k in range(3, truncation_size + 1):
        # Вычисление delta_{d,k-1} по рекуррентной формуле
        delta_k_minus_1 = delta[-1] * (k - 2 - d) / (k - 1)
        delta.append(delta_k_minus_1)
        
        # Вычисление lambda_k по рекуррентной формуле
        term = ((k - 1 - d) / k - phi1) * delta_k_minus_1
        lambda_k = beta1 * lambdas[-1] + term
        lambdas.append(lambda_k)
    
    return lambdas[:truncation_size]  # Возвращаем только запрошенное количество элементов


In [3]:

# Пример использования
d = 0.5
phi1 = 0.2
beta1 = 0.3
truncation_size = 5


In [4]:
lambda_sequence = compute_lambda_sequence(d, phi1, beta1, truncation_size)
for k, lambda_k in enumerate(lambda_sequence, start=1):
    print(f"lambda_{k} = {lambda_k}")

lambda_1 = 0.4
lambda_2 = 0.145
lambda_3 = 0.08099999999999999
lambda_4 = 0.05086249999999999
lambda_5 = 0.034789999999999995


In [5]:
def fit_lambda_parameters(target_lambdas, initial_guess=None, bounds=None):
    """
    Находит параметры d, phi1, beta1, которые наилучшим образом соответствуют заданным lambda_k.
    
    Параметры:
    target_lambdas : list
        Список целевых значений lambda_k (начиная с lambda_1).
    initial_guess : tuple (d, phi1, beta1), optional
        Начальное предположение для параметров.
    bounds : tuple of tuples, optional
        Границы для параметров: ((d_min, phi1_min, beta1_min), (d_max, phi1_max, beta1_max)).
    
    Возвращает:
    dict
        Словарь с найденными параметрами и информацией об оптимизации.
    """
    if initial_guess is None:
        initial_guess = (0.5, 0.5, 0.5)  # Стандартное начальное предположение
    if bounds is None:
        bounds = ((0, -np.inf, 0), (1, np.inf, np.inf))  # d ∈ (0,1), beta1 ≥ 0
    
    def residuals(params):
        d, phi1, beta1 = params
        computed_lambdas = compute_lambda_sequence(d, phi1, beta1, len(target_lambdas))
        return np.array(computed_lambdas) - np.array(target_lambdas)
    
    result = least_squares(
        residuals,
        x0=initial_guess,
        bounds=bounds,
        method='trf'  # Метод, поддерживающий границы
    )
    
    return result.x[0], result.x[1], result.x[2]

In [6]:
fit_lambda_parameters(lambda_sequence)

(0.4999999984665524, 0.19999999735284912, 0.2999999958131293)